In [1]:
# Nativos
import random as rn
import os
import sys
import gc
import time

#calculo
import numpy as np
import pandas as pd
import scipy

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

#warning ignore future
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
gc.collect()

SEED = int(time.time() / 1000)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

from utils_alertas import *

EJEMPLO:
202104 mes cerrado de la cosecha de 202104 [202011, 202012, 202101, 202102, 202103, 202104] [202102, 202103, 202104] 
 historia 
 [202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104] 
 ---------------------------------------------------------------------------------------------------- 
 [202105, 202106] 
 ---------------------------------------------------------------------------------------------------- 
 [202005, 202006]


In [2]:
data = pd.read_csv('AVANCE_6.csv', dtype={
    'COD_UNICO': str,
    'PERIODO': str
})
data.head()

,ADEX_PROM_U12M_ACTUAL_IMP_OVER_12M_PASADO_MENOR_0.5,_aux_TRX_SUBIDA_30PERCENT_TRANSFERENCIAS__CASH_OUT_MES_ANT,_75_cut_up_ROTACION_CTASXCOBRAR_VAR_DIAS_CEF_SITUACION,FUNNEL_ALERTA,_aux_TRX_CAIDA_10PERCENT_OPERACIONES_MESA__MES_ANT,DEBT_IN_SF_ULT_RCC_x,_0_cut_up_TENDENCIA_PERCENT_NO_CLASIF_NORMAL_GRUPO_ECON_U7M,TIENE_IMPORTE_TRIB_U1M_OVER_U12M_MAYOR_1,ADEX_MAYOR_U3M_IMP,ACTIVIDAD__Comercio,...,_5_cut_up_NUM_MESES_SIN_LINEA_VIGENTE,_2_cut_up_NUM_DOCS_PROT_PROMEDIO_U9M,_1_cut_up_NUM_DOCS_PROT_U6M,_1_cut_up_NUM_DOCS_TRIB_U12M,_2_cut_up_NUM_DOCS_IMPAG_PROMEDIO_U6M,_3_cut_up_NUM_DOCS_IMPAG_U12M,_1_cut_up_NUM_DOCS_LAB_U6M,_1_cut_up_NUM_DOCS_TRIB_PROMEDIO_U9M,cut_letal_SUBSECTOR_s2_provincia,_2_cut_up_NUM_DOCS_IMPAG_PROMEDIO_U9M
0,0.0,0.0,1.0,NaN,0.0,9253118.94,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,NaN,0.0,521627.95,0.0,0.0,1.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,NaN,0.0,48350156.96,0.0,0.0,1.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,NaN,0.0,24993499.25,0.0,0.0,0.0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,NaN,0.0,973224.29,0.0,0.0,1.0,0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [3]:
'COD_UNICO' in data.columns, 'PERIODO' in data.columns, 

(True, True)

In [4]:
print(data.shape)
data = filtro_banca_gran(data)
print(data.shape)

(24014, 3638)
(2488, 3638)


In [5]:
pd.crosstab(
    data['PERIODO'], data['ZONAL']
)

ZONAL,BEL ZONAL 1,BEL ZONAL 3
PERIODO,,
202009,215,203
202010,214,203
202011,215,200
202012,216,200
202101,212,198
202102,212,200


### CREACION DE COLUMNAS A MEZCLAR

In [6]:
cols_combine = [col for col in data.columns if set(data[col].unique()) == {1, 0} if '1_and_1' not in col]
print(len(cols_combine))

2344


In [7]:
cols_combine = \
    [col for col in cols_combine if 'SCOTIA' not in col.upper() \
                                 and 'BCP' not in col.upper() \
                                 and 'BBVA' not in col.upper() \
                                 and 'BIF' not in col.upper() \
                                 and '%' not in col.upper() \
                                 and 'COVID' not in col.upper() \
                                 and 'REFIN' not in col.upper() 
    ]
print(len(cols_combine))

2259


In [8]:
for col in ['cambio_seguir_covid'] + [col for col in data.columns if 'target' in col.lower()]:
    if col in cols_combine:
        cols_combine.remove(col)

len(cols_combine)

2258

### GENERACION DE ALERTAS

In [9]:
existentes = pd.read_csv('COMBINACIONES_GRAN_EMPRESA_LIMA.csv', sep=',').sort_values(by=['SCORE'], ascending=False)[
    ['VAR_1', 'VAR_2', 'VAR_3', 'CASOS', 'CORRECTOS', 'SCORE']
]

existentes.head(50)

,,,,,VAR_1,VAR_2,VAR_3,CASOS,CORRECTOS,SCORE
_aux_TRX_CAIDA_30PERCENT_POS__U6M,_2_cut_up_NUM_BANCOS_NOIBK_DEBT_SUBIDA_SEGUIDA_DE_U3M,NaN,6,2,0.3333333333333333_aux_TRX_CAIDA_50PERCENT_DEP...,TIENE_1_O_MAS_GRUPO_ECON_VPC_NO_CLASIF_NORMAL,_0.2_cut_down_minimo_percent_sow_ibk_u12m,7.0,3.0,0.428571
_aux_CASHOUT_CAIDA_30PERCENT_PAGO_DE_SERVICIOS__U12M,IMPORTE_IMPAG_REGISTRADO_MAYOR_A_MIL_SOLES,_1.19_cut_up_ROTACION_CTASXCOBRAR_VAR_CEF_SITUACION,10,5,0.5,NaN,NaN,NaN,NaN,NaN
_aux_CASHOUT_CAIDA_20PERCENT_OPERACIONES_MESA__U12M,sei_letal_gran_empresa,NaN,11,5,0.45454545454545453,NaN,NaN,NaN,NaN,NaN
_aux_TRX_CAIDA_20PERCENT_TRANSFERENCIAS__U6M,_5_cut_up_NUM_MESES_CON_ATRASO_U6M_COLDIRECT_SF,NaN,15,8,0.5333333333333333,NaN,NaN,NaN,NaN,NaN
_1_cut_up_NUM_BANCOS_NOIBK_DEBT_MAYOR_DE_U3M,_5_cut_up_NUM_MESES_CON_ATRASO_U6M_COLDIRECT_SF,NaN,13,7,0.5384615384615384,NaN,NaN,NaN,NaN,NaN
_aux_TIENE_TRX_POS__U12M,_0.3_cut_down_minimo_percent_auto_hipo_pren_u12m,NaN,11,6,0.5454545454545454,NaN,NaN,NaN,NaN,NaN
TIENE_MAS_DE_1_MES_SIN_PAGOS_MASIVOS__CASHOUT,_0.3_cut_down_maximo_percent_ibk_u12m,_aux_TRX_CAIDA_30PERCENT_OTROS_NO_TABLERO_MES_ANT,7,3,0.42857142857142855,NaN,NaN,NaN,NaN,NaN
_0.1_cut_down_minimo_percent_auto_hipo_u12m,_3_cut_up_NUM_MESES_CON_ATRASO_U6M_COLDIRECT_SF,NaN,13,6,0.46153846153846156,NaN,NaN,NaN,NaN,NaN
_4_cut_up_MAX_DIAS_ATRASO_SFSF_ULT_RCC,TIENE_DISMINUCION_PAGO_PROVEEDORES_60PERCENT_A_MAS_U7M,NaN,7,3,0.42857142857142855,NaN,NaN,NaN,NaN,NaN
_1_cut_up_NUM_MESES_SIN_LINEA_VIGENTE,_aux_TRX_CAIDA_10PERCENT_POS__U11M,NaN,7,4,0.5714285714285714,NaN,NaN,NaN,NaN,NaN


In [10]:
#existentes.to_csv('COMBINACIONES_GRAN_EMPRESA_LIMA.csv', index=False)

In [11]:
try:
    existentes['union'] = existentes[['VAR_1', 'VAR_2', 'VAR_3']].apply(
        lambda _: '1_and_1_{}___{}'.format(_[0], _[1]) if pd.isnull(_[2]) else '1_and_1_and_1_{}___{}____{}'.format(_[0], _[1], _[2]), 
        axis=1
    )
    listado = list(existentes['union'].unique())
    len(listado)
except:
    listado = []
    
len(listado)

51

In [12]:
import random as rn
import itertools

col_target = 'target_FINAL_fusion_atraso_feve_2Mbueno'
display(pd.crosstab(
    data[col_target].fillna(-1), data['PERIODO']
))

aucc = {}
struct = []

data = data[data[col_target].notnull()][cols_combine + ['COD_UNICO', col_target, 'PERIODO']]
data.shape

PERIODO,202009,202010,202011,202012,202101,202102
target_FINAL_fusion_atraso_feve_2Mbueno,,,,,,
-1.0,140,127,119,109,109,106
0.0,256,272,281,288,286,287
1.0,22,18,15,19,15,19


(1778, 3638)

In [13]:
SEED = int(time.time() / 1000)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

rn.shuffle(cols_combine)

In [ ]:
from datetime import datetime
tiempo_inicio = time.time()
nn = len(struct)

while True:
    x = rn.choice(cols_combine)
    y = rn.choice(cols_combine)
    z = rn.choice(cols_combine)
    
    col_evaluate = '1_and_1_and_1_{}___{}____{}'.format(x, y, z)
    
    if col_evaluate not in listado and x != y and x!= z and y!= z:
        listado.append(col_evaluate)
        
        lambda_basico = lambda _:  _[0] >= 1 and _[1] >= 1 and _[2] >= 1
        data[col_evaluate] = data[[x, y, z]].apply(lambda_basico, axis=1).astype(int)
        #data_play_1[col_evaluate] = data_play_1[[x, y, z]].apply(lambda_basico, axis=1).astype(int)
        
        
        coincidencias = len(set(data[data[col_evaluate] == 1]['COD_UNICO'].unique()))
        acertados = len(set(data[(data[col_evaluate] == 1) & (data[col_target] == 1)]['COD_UNICO'].unique()))
        
        if coincidencias > 5 and acertados/coincidencias > 0.4:
            aucc[col_evaluate] = acertados/coincidencias
            struct.append(
                [x, y, z, coincidencias, acertados, acertados/coincidencias]
            )
            tiempo_inicio = time.time()
            nn = len(struct)
            #print(struct[-1])
            
        ### Generado pares de 2 
        for a, b in list(itertools.combinations([x, y, z], 2)):
            
            col_evaluate = '1_and_1_{}___{}'.format(a, b)
            if col_evaluate not in listado:
                lambda_basico = lambda _:  _[0] >= 1 and _[1] >= 1
                data[col_evaluate] = data[[a, b]].apply(lambda_basico, axis=1).astype(int)
                #data_play_1[col_evaluate] = data_play_1[[a, b]].apply(lambda_basico, axis=1).astype(int)
                
                coincidencias = len(set(data[data[col_evaluate] == 1]['COD_UNICO'].unique()))
                acertados = len(set(data[(data[col_evaluate] == 1) & (data[col_target] == 1)]['COD_UNICO'].unique()))
                
                if coincidencias > 5 and acertados/coincidencias > 0.4:
                    aucc[col_evaluate] = acertados/coincidencias
                    struct.append(
                        [a, b, None, coincidencias, acertados, acertados/coincidencias]
                    )
                    tiempo_inicio = time.time()
                    nn = len(struct)
                    #print(struct[-1])
        
        if len(struct) >= 5:
            pd.DataFrame(struct).to_csv('COMBINACIONES_GRAN_EMPRESA_LIMA.csv', header=False, index=False, mode='a')
            print("/"*25, datetime.now())
            struct = []
            
        if time.time() - tiempo_inicio >= 3600 and nn == len(struct):
            print("Mas de 1 hora sin nueva combinacion de alerta")
            break
                                                        

///////////////////////// 2021-05-28 05:02:56.135952
///////////////////////// 2021-05-28 05:03:24.168627
///////////////////////// 2021-05-28 05:04:26.033039
///////////////////////// 2021-05-28 05:04:43.524256
///////////////////////// 2021-05-28 05:04:52.719693
///////////////////////// 2021-05-28 05:05:41.566591
///////////////////////// 2021-05-28 05:06:03.137542
///////////////////////// 2021-05-28 05:07:07.772698
///////////////////////// 2021-05-28 05:07:34.117342
///////////////////////// 2021-05-28 05:08:10.635991
///////////////////////// 2021-05-28 05:10:21.375936
///////////////////////// 2021-05-28 05:12:12.358001
///////////////////////// 2021-05-28 05:12:41.884942
///////////////////////// 2021-05-28 05:13:19.729399
///////////////////////// 2021-05-28 05:14:17.276684
///////////////////////// 2021-05-28 05:15:04.792055
///////////////////////// 2021-05-28 05:15:48.524071
///////////////////////// 2021-05-28 05:16:12.420343
///////////////////////// 2021-05-28 05:17:03.

In [ ]:
#pd.DataFrame([], columns=['VAR_1', 'VAR_2', 'VAR_3', 'CASOS', 'CORRECTOS', 'SCORE']).to_csv('COMBINACIONES_MEDIANA_LIMA.csv', index=False)
#pd.DataFrame([], columns=['VAR_1', 'VAR_2', 'VAR_3', 'CASOS', 'CORRECTOS', 'SCORE']).to_csv('COMBINACIONES_GRAN_EMPRESA_LIMA.csv', index=False)
#pd.DataFrame([], columns=['VAR_1', 'VAR_2', 'VAR_3', 'CASOS', 'CORRECTOS', 'SCORE']).to_csv('COMBINACIONES_S1_PROVINCIA.csv', index=False)
#pd.DataFrame([], columns=['VAR_1', 'VAR_2', 'VAR_3', 'CASOS', 'CORRECTOS', 'SCORE']).to_csv('COMBINACIONES_S2_PROVINCIA.csv', index=False)